In [104]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [105]:
df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
combine = [df, test_df]

In [106]:
null_cols = df.isnull().sum().where(df.isnull().sum() > 0)[df.isnull().sum().where(df.isnull().sum() > 0) > 0] # there has to be a better way but im js like that 
null_cols

LotFrontage      259.0
Alley           1369.0
MasVnrType       872.0
MasVnrArea         8.0
BsmtQual          37.0
BsmtCond          37.0
BsmtExposure      38.0
BsmtFinType1      37.0
BsmtFinType2      38.0
Electrical         1.0
FireplaceQu      690.0
GarageType        81.0
GarageYrBlt       81.0
GarageFinish      81.0
GarageQual        81.0
GarageCond        81.0
PoolQC          1453.0
Fence           1179.0
MiscFeature     1406.0
dtype: float64

In [107]:
# data cleaning
null_cols_80 = null_cols[null_cols > len(df.index) * 0.8].index.values # re check how many %
null_cols_80 = [n.strip() for n in null_cols_80]
df = df.drop(null_cols_80, axis=1)

df['LotFrontage'] = df['LotFrontage'].fillna(0)
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)
df['BsmtFinSF1'] = df['BsmtFinSF1'].fillna(0)
df['BsmtFinSF2'] = df['BsmtFinSF2'].fillna(0)
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)
df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode)

In [108]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   str    
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   str    
 6   LotShape       1460 non-null   str    
 7   LandContour    1460 non-null   str    
 8   Utilities      1460 non-null   str    
 9   LotConfig      1460 non-null   str    
 10  LandSlope      1460 non-null   str    
 11  Neighborhood   1460 non-null   str    
 12  Condition1     1460 non-null   str    
 13  Condition2     1460 non-null   str    
 14  BldgType       1460 non-null   str    
 15  HouseStyle     1460 non-null   str    
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuilt      1460

In [109]:
df.isnull().sum().where(df.isnull().sum() > 0)[df.isnull().sum().where(df.isnull().sum() > 0) > 0] # there has to be a better way but im js like that 

MasVnrType      872.0
BsmtQual         37.0
BsmtCond         37.0
BsmtExposure     38.0
BsmtFinType1     37.0
BsmtFinType2     38.0
FireplaceQu     690.0
GarageType       81.0
GarageFinish     81.0
GarageQual       81.0
GarageCond       81.0
dtype: float64

In [110]:
categorical_cols = df.select_dtypes(include=['object', 'str'])
numerical_cols = df.select_dtypes(include=['int', 'float']).columns.drop(['Id'])
to_scale_numerical_cols = numerical_cols.drop(['MSSubClass', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFullBath' ,'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars',  'MoSold', 'YrSold', 'SalePrice'])

In [123]:
MSSubClass_mapping = {
    20: 0,
    30: 1,
    40: 2,
    45: 3,
    50: 4,
    60: 5,
    70: 6,
    75: 7,
    80: 8,
    85: 9,
    90: 10,
    120: 11,
    150: 12,
    160: 13,
    180: 14,
    190: 15
}

Street_mapping = {
    "Grvl": 0,
    "Pave": 1
}

LandSlope_mapping = {
    "Sev": 0,
    "Mod": 1,
    "Gtl": 2
}

ExterQual_mapping = {
    "Po": 0,
    "Fa": 1,
    "TA": 2,
    "Gd": 3,
    "Ex": 4
}

ExterCond_mapping = {
    "Po": 0,
    "Fa": 1,
    "TA": 2,
    "Gd": 3,
    "Ex": 4
}

BsmtQual_mapping = {
    "NA": 0,
    "Po": 1,
    "Fa": 2,
    "TA": 3,
    "Gd": 4,
    "Ex": 5
}

BsmtCond_mapping = {
    "NA": 0,
    "Po": 1,
    "Fa": 2,
    "TA": 3,
    "Gd": 4,
    "Ex": 5
}

BsmtExposure_mapping = {
    "NA": 0,
    "No": 1,
    "Mn": 2,
    "Av": 3,
    "Gd": 4
}

BsmtFinType1_mapping = {
    "NA": 0,
    "Unf": 1,
    "LwQ": 2,
    "Rec": 3,
    "BLQ": 4,
    "ALQ": 5,
    "GLQ": 6
}

BsmtFinType2_mapping = BsmtFinType1_mapping.copy()

HeatingQC_mapping = {
    "Po": 0,
    "Fa": 1,
    "TA": 2,
    "Gd": 3,
    "Ex": 4
}

CentralAir_mapping = {
    "N": 0,
    "Y": 1
}

KitchenQual_mapping = {
    "Po": 0,
    "Fa": 1,
    "TA": 2,
    "Gd": 3,
    "Ex": 4
}

FireplaceQu_mapping = {
    "NA": 0,
    "Po": 1,
    "Fa": 2,
    "TA": 3,
    "Gd": 4,
    "Ex": 5
}

GarageFinish_mapping = {
    "NA": 0,
    "Unf": 1,
    "RFn": 2,
    "Fin": 3
}

GarageQual_mapping = {
    "NA": 0,
    "Po": 1,
    "Fa": 2,
    "TA": 3,
    "Gd": 4,
    "Ex": 5
}

GarageCond_mapping = {
    "NA": 0,
    "Po": 1,
    "Fa": 2,
    "TA": 3,
    "Gd": 4,
    "Ex": 5
}

PavedDrive_mapping = {
    "N": 0,
    "P": 1,
    "Y": 2
}

mappings = {
    "MSSubClass": MSSubClass_mapping,
    "Street": Street_mapping,
    "LandSlope": LandSlope_mapping,
    "ExterQual": ExterQual_mapping,
    "ExterCond": ExterCond_mapping,
    "BsmtQual": BsmtQual_mapping,
    "BsmtCond": BsmtCond_mapping,
    "BsmtExposure": BsmtExposure_mapping,
    "BsmtFinType1": BsmtFinType1_mapping,
    "BsmtFinType2": BsmtFinType2_mapping,
    "HeatingQC": HeatingQC_mapping,
    "CentralAir": CentralAir_mapping,
    "KitchenQual": KitchenQual_mapping,
    "FireplaceQu": FireplaceQu_mapping,
    "GarageFinish": GarageFinish_mapping,
    "GarageQual": GarageQual_mapping,
    "GarageCond": GarageCond_mapping,
    "PavedDrive": PavedDrive_mapping,
}

mapped_col = []
for dataset in combine:
    for col, mapping in mappings.items():
        mapped_col.append(col)
        dataset[col] = dataset[col].map(mapping)
        # df[col] = df[col].map(mapping).fillna(0)


df.isnull().sum().where(df.isnull().sum() > 0)[df.isnull().sum().where(df.isnull().sum() > 0) > 0] # there has to be a better way but im js like that 

MasVnrType      872.0
BsmtQual         37.0
BsmtCond         37.0
BsmtExposure     38.0
BsmtFinType1     37.0
BsmtFinType2     38.0
FireplaceQu     690.0
GarageType       81.0
GarageFinish     81.0
GarageQual       81.0
GarageCond       81.0
dtype: float64

In [112]:
# df.info()

In [113]:
# one hot 
df['Electrical'] = df['Electrical'].astype('str')

not_mapped = [item for item in categorical_cols if item not in mapped_col]

for dataset in combine:
    for col in not_mapped:
        one_hot = pd.get_dummies(dataset[col], dtype='int')
        dataset = dataset.drop(col, axis=1)
        dataset = pd.concat([dataset, one_hot], axis=1)

In [114]:
# df[numerical_cols].info()

In [115]:
train = combine[0]
test = combine[1]

_, yearbuilt_bins = pd.qcut(train['YearBuilt'], 4, retbins=True)

for dataset in combine:
    dataset['YearBuilt'] = pd.cut(dataset['YearBuilt'],
                                   bins=yearbuilt_bins,
                                   labels=[0, 1, 2, 3],
                                   include_lowest=True).astype(int)

_, yearremod_bins = pd.qcut(train['YearRemodAdd'], 4, retbins=True)

for dataset in combine:
    dataset['YearRemodAdd'] = pd.cut(dataset['YearRemodAdd'],
                                     bins=yearremod_bins,
                                     labels=[0, 1, 2, 3],
                                     include_lowest=True).astype(int)

_, garage_bins = pd.qcut(train['GarageYrBlt'].dropna(), 4, retbins=True)

for dataset in combine:
    dataset['GarageYrBlt'] = pd.cut(dataset['GarageYrBlt'],
                                    bins=garage_bins,
                                    labels=[0, 1, 2, 3],
                                    include_lowest=True).astype('Int64')


In [122]:
df.isnull().sum().where(df.isnull().sum() > 0)[df.isnull().sum().where(df.isnull().sum() > 0) > 0] # there has to be a better way but im js like that 

MasVnrType      872.0
BsmtQual         37.0
BsmtCond         37.0
BsmtExposure     38.0
BsmtFinType1     37.0
BsmtFinType2     38.0
FireplaceQu     690.0
GarageType       81.0
GarageFinish     81.0
GarageQual       81.0
GarageCond       81.0
dtype: float64

In [116]:
df[to_scale_numerical_cols]

,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal
0,65.0,8450,196.0,706,0,150,856,856,854,0,1710,548,0,61,0,0,0,0,0
1,80.0,9600,0.0,978,0,284,1262,1262,0,0,1262,460,298,0,0,0,0,0,0
2,68.0,11250,162.0,486,0,434,920,920,866,0,1786,608,0,42,0,0,0,0,0
3,60.0,9550,0.0,216,0,540,756,961,756,0,1717,642,0,35,272,0,0,0,0
4,84.0,14260,350.0,655,0,490,1145,1145,1053,0,2198,836,192,84,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,62.0,7917,0.0,0,0,953,953,953,694,0,1647,460,0,40,0,0,0,0,0
1456,85.0,13175,119.0,790,163,589,1542,2073,0,0,2073,500,349,0,0,0,0,0,0
1457,66.0,9042,0.0,275,0,877,1152,1188,1152,0,2340,252,0,60,0,0,0,0,2500
1458,68.0,9717,0.0,49,1029,0,1078,1078,0,0,1078,240,366,0,112,0,0,0,0


In [117]:
# scale
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

for dataset in combine:    
    dataset[to_scale_numerical_cols] = scaler.fit_transform(dataset[to_scale_numerical_cols])

In [118]:
# split
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [119]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

rf = RandomForestRegressor()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

lr_rmse = np.sqrt(mean_squared_error(y_test, lr_pred))
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))

lr_mse = mean_squared_error(y_test, lr_pred)
rf_mse = mean_squared_error(y_test, rf_pred)

print(f"lr mse: {lr_mse:.3f}\n rf mse : {rf_mse:.3f}")
print(f"lr rmse: {lr_rmse:.3f}\n rf rmse : {rf_rmse:.3f}")

ValueError: Input contains NaN

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# log-transform target
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

lr = LinearRegression()
lr.fit(X_train, y_train_log)
lr_pred_log = lr.predict(X_test)

rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train_log)
rf_pred_log = rf.predict(X_test)

lr_rmse = np.sqrt(mean_squared_error(y_test_log, lr_pred_log))
rf_rmse = np.sqrt(mean_squared_error(y_test_log, rf_pred_log))

lr_mse = mean_squared_error(y_test_log, lr_pred_log)
rf_mse = mean_squared_error(y_test_log, rf_pred_log)

print(f"lr mse: {lr_mse:.5f}\n rf mse : {rf_mse:.5f}")
print(f"lr rmse: {lr_rmse:.5f}\n rf rmse : {rf_rmse:.5f}")


lr mse: 0.03817
 rf mse : 0.02442
lr rmse: 0.19538
 rf rmse : 0.15628
